<a href="https://colab.research.google.com/github/Huangjian2013/ai-demo/blob/main/code/03-Swarm-sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/swarm.git --quiet


In [8]:
import openai
from google.colab import userdata
from swarm import Swarm, Agent


openai.api_key = userdata.get('REAL_OPENAI_KEY')

In [18]:
client = Swarm(client=openai)


def process_refund():
    """执行退款操作."""
    print(f"[mock] 退款操作...")
    return "成功!"


def apply_discount():
    """给用户打折."""
    print("[mock] 正在打折...")
    return "帮用户的商品打了9折"


triage_agent = Agent(
    name="中转智能体",
    instructions="确认用户的操作是什么，转移到对应的智能体去处理.",
)
sales_agent = Agent(
    name="销售智能体",
    instructions="非常热情地给用户介绍蜜蜂.",
)
refunds_agent = Agent(
    name="退款智能体",
    instructions="如果用户乐意，给用户提供折扣。否则走退款逻辑",
    functions=[process_refund, apply_discount],
)


def transfer_back_to_triage():
    """如果用户的回答和当前智能体没什么关系，转移到中转智能体去执行."""
    print("[转移智能体操作] --> triage_agent")
    return triage_agent


def transfer_to_sales():
    print("[转移智能体操作] --> sales_agent")
    return sales_agent


def transfer_to_refunds():
    print("[转移智能体操作] --> refunds_agent")
    return refunds_agent


triage_agent.functions = [transfer_to_sales, transfer_to_refunds]
sales_agent.functions.append(transfer_back_to_triage)
refunds_agent.functions.append(transfer_back_to_triage)

messages = []
while True:
    user = input("输入: ")
    if (user == "exit"): break
    messages.append({"role": "user", "content": user})

    response = client.run(
        agent=triage_agent,
        messages=messages,
    )

    print(response.messages[-1]["content"])

输入: 我要买蜜蜂
[转移智能体操作] --> sales_agent
蜜蜂是一种神奇而有趣的小生物！让我们来聊聊它们吧。

蜜蜂不仅是辛勤的授粉者，还负责生产我们喜爱的蜂蜜。它们生活在蜜蜂群中，由一只蜂后、许多工蜂和少数雄蜂组成。每只小蜜蜂都有自己独特的职责，例如采集花蜜、保护蜂巢和哺育幼蜂。

更有趣的是，蜂蜜不仅美味，而且富含多种营养，小小的一罐蜂蜜蕴藏着大自然的精华。无论是在早餐时用来涂抹面包，还是在茶中调和甜味，蜂蜜总能为你的生活增添一份甜美。

如果真的有兴趣购买蜜蜂，一开始可能需要学习一些基本的养蜂知识，比如如何建设蜂巢、管理蜂群和收蜜等。这将是一个令人着迷且充满回报的过程。

希望你和这些勤劳的小伙伴们相处愉快，有任何疑问都可以随时问我哦！🐝🍯
输入: 我要退款
[转移智能体操作] --> sales_agent
[转移智能体操作] --> refunds_agent
[mock] 退款操作...
您的退款已经成功处理。
输入: exit
